### Fine - Tuning BERT for Seque

The model will be finetuned to classify sequences as either positive or negative. In order to train the model we need to:

<ol>

<li>Retrive dataset.</li>
<li>Tokenize and pre-process the dataset using the collator function.</li>
<li>Instantiate the trainer and begin the training loop.</li>
<li>Conduct inference.</li>
</ol>

In [ ]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer

In [ ]:
!pip install datasets evaluate

In [ ]:
from datasets import load_dataset

In [ ]:
checkpoint = "google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
raw_dataset = load_dataset("glue", "sst2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
raw_dataset["train"].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [ ]:
def tokenize(example):
  return tokenizer(example["sentence"], truncation=True)

In [ ]:
tokenized_dataset = raw_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
train_arguments = TrainingArguments("test-directory")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_predict):
  metric = evaluate.load("glue", "sst2")
  logits, labels = eval_predict
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [ ]:
trainer = Trainer(
    model,
    args = train_arguments,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss
500,0.432300
1000,0.384400
1500,0.375700
2000,0.387000
2500,0.340100
3000,0.336500
3500,0.323800
4000,0.317800
4500,0.318200
5000,0.312400


TrainOutput(global_step=25257, training_loss=0.23319151014775144, metrics={'train_runtime': 1322.3203, 'train_samples_per_second': 152.797, 'train_steps_per_second': 19.101, 'total_flos': 3251945755383360.0, 'train_loss': 0.23319151014775144, 'epoch': 3.0})

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [ ]:
import torch
print(torch.version.cuda)

12.1


In [ ]:
pip install torch --no-cache-dir

In [ ]:
import os
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [ ]:
trainer.predict(tokenized_dataset["test"], ignore_keys=True)

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
